In [ ]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statannot
from functools import reduce 
import matplotlib.gridspec as gridspec

chr_size = {'chr':['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                   'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20',
                   'chrX'],
            'size':[228955476,192909870,193512962,173232586,190271649,181913984,172638558,147524517,133925568,96855220,
                    138871270,132992465,111518885,132195074,113086136,82009341,97546542,75807056,60553053,78739555,
                    153601299]
           } 

chr_size = pd.DataFrame(chr_size)



os.system('wget https://hgdownload.soe.ucsc.edu/goldenPath/macFas5/bigZips/genes/macFas5.ncbiRefSeq.gtf.gz -P ./source_data/reference')
os.system('gunzip ./source_data/reference/macFas5.ncbiRefSeq.gtf.gz')

os.system('wget https://hgdownload.soe.ucsc.edu/goldenPath/macFas5/bigZips/genes/macFas5.refGene.gtf.gz -P ./source_data/reference')
os.system('gunzip ./source_data/reference/macFas5.refGene.gtf.gz')

os.system('wget https://hgdownload.soe.ucsc.edu/goldenPath/macFas5/database/rmsk.txt.gz -P ./source_data/reference')
os.system('gunzip ./source_data/reference/rmsk.txt.gz')


file = './source_data/reference/macFas5.ncbiRefSeq.gtf'
ncbiRefSeq= pd.read_csv(file, sep = '\t', names=['chr','ncbiRefSeq','feature','start','end','A','strand','B','INFO'])

file = './source_data/reference/macFas5.refGene.gtf'
refGene= pd.read_csv(file, sep = '\t', names=['chr','refGene','feature','start','end','A','strand','B','INFO'])

file = './source_data/reference/rmsk.txt'
rmsk= pd.read_csv(file, sep = '\t', names=['score','a','b','c','d','chr','start','end','e','strand','sub3','TEs','sub2','f','g','h','i'])

####DEG

file = './source_data/DETE/TE_group1_padj_score.txt'
DET_G1 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TE_group2_padj_score.txt'
DET_G2 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TE_group3_padj_score.txt'
DET_G3 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TE_group4_padj_score.txt'
DET_G4 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TE_group5_padj_score.txt'
DET_G5 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TE_group6_padj_score.txt'
DET_G6 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEsum_group1_padj_score.txt'
DETsum_G1 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEsum_group2_padj_score.txt'
DETsum_G2 = pd.read_csv(file, sep = '\t', header = 0)



file = './source_data/DETE/TEtranscript_group1_padj_score.txt'
DETT_G1 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEtranscript_group2_padj_score.txt'
DETT_G2 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEtranscript_group3_padj_score.txt'
DETT_G3 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEtranscript_group4_padj_score.txt'
DETT_G4 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEtranscript_group5_padj_score.txt'
DETT_G5 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEtranscript_group6_padj_score.txt'
DETT_G6 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEtranscript_group7_padj_score.txt'
DETT_G7 = pd.read_csv(file, sep = '\t', header = 0)

file = './source_data/DETE/TEtranscript_group8_padj_score.txt'
DETT_G8 = pd.read_csv(file, sep = '\t', header = 0)


file = './source_data/DETE/batch_cor_TE_LRT_normalized_count_TC.txt'
#file = './source_data/batch_correction/batch_cor_TECount_filled_mfa.txt'
TE_norm = pd.read_csv(file, sep = '\t', header = 0)
TE_norm = TE_norm.reset_index(drop=False)
TE_norm = TE_norm.rename(columns={'index':'TE_ID'})
regex_pat = re.compile(r'[|:+-]')
TE_norm['TE_ID'] = TE_norm['TE_ID'].str.replace(regex_pat,'.',regex=True)



file = './source_data/DETE/batch_cor_TEtranscripts_LRT_normalized_count_ELTC.txt'
TEtr_norm = pd.read_csv(file, sep = '\t', header = 0)
TEtr_norm = TEtr_norm.reset_index(drop=False)
TEtr_norm = TEtr_norm.rename(columns={'index':'TE_ID'})
TEtr_norm['gene'] = TEtr_norm.TE_ID.str.split(':').str[0]
TEtr_norm['genus'] = TEtr_norm.TE_ID.str.split(':').str[1]
TEtr_norm['family'] = TEtr_norm.TE_ID.str.split(':').str[2]
TEtr_norm = TEtr_norm.iloc[:,1:]

DET_G1['GROUP'] = 'G1'
DET_G2['GROUP'] = 'G2'
DET_G3['GROUP'] = 'G3'
DET_G4['GROUP'] = 'G4'
DET_G5['GROUP'] = 'G5'
DET_G6['GROUP'] = 'G6'
DET_ALL = pd.concat([DET_G1,DET_G2,DET_G3,DET_G4,DET_G5,DET_G6],axis=0)


DETsum_G1['GROUP'] = 'G1'
DETsum_G2['GROUP'] = 'G2'
DETsum_ALL = pd.concat([DETsum_G1,DETsum_G2],axis=0)


DETT_G1['GROUP'] = 'G1'
DETT_G2['GROUP'] = 'G2'
DETT_G3['GROUP'] = 'G3'
DETT_G4['GROUP'] = 'G4'
DETT_G5['GROUP'] = 'G5'
DETT_G6['GROUP'] = 'G6'
DETT_G7['GROUP'] = 'G7'
DETT_G8['GROUP'] = 'G8'
DETT_ALL = pd.concat([DETT_G1,DETT_G2,DETT_G3,DETT_G4,DETT_G5,DETT_G6,DETT_G7,DETT_G8],axis=0)

DET_G24 = pd.concat([DET_G2,DET_G4],axis=0)

## Fig. 7b ; Extended Data Fig. 9c

In [ ]:
from scipy import stats 
from sklearn.metrics import mean_absolute_error as score_MAE
from sklearn.metrics import mean_squared_error as score_MSE 
from sklearn.metrics import median_absolute_error as score_MED
from sklearn.metrics import r2_score

In [ ]:
def file_parsing(data_melt):

    conditionlist = [data_melt.variable == 'A_FST',
                     data_melt.variable == 'B_FST',
                     data_melt.variable == 'A_SND',
                     data_melt.variable == 'B_SND',
                     data_melt.variable == 'A_TRD',
                     data_melt.variable == 'B_TRD',
                 
                     data_melt.variable == 'C_FST',
                     data_melt.variable == 'D_FST',
                     data_melt.variable == 'C_SND',
                     data_melt.variable == 'D_SND',
                     data_melt.variable == 'C_TRD',
                     data_melt.variable == 'D_TRD',
                 
                     data_melt.variable == 'E_FST',
                     data_melt.variable == 'F_FST',
                     data_melt.variable == 'E_SND',
                     data_melt.variable == 'F_SND',
                     data_melt.variable == 'E_TRD',
                     data_melt.variable == 'F_TRD',
                 
                     data_melt.variable == 'G_FST',
                     data_melt.variable == 'H_FST',
                     data_melt.variable == 'G_SND',
                     data_melt.variable == 'H_SND',
                     data_melt.variable == 'G_TRD',
                     data_melt.variable == 'H_TRD']
    choicelist = [3,3,4,4,5,5,7,7,8,8,9,9,12,12,13,13,14,14,16,16,17,17,18,18]
    data_melt['age'] = np.select(conditionlist,choicelist, default='Not Specified')


    conditionlist = [data_melt.variable.str.split('_').str[0] == 'A',
                     data_melt.variable.str.split('_').str[0] == 'B',
                     data_melt.variable.str.split('_').str[0] == 'C',
                     data_melt.variable.str.split('_').str[0] == 'D',
                     data_melt.variable.str.split('_').str[0] == 'E',
                     data_melt.variable.str.split('_').str[0] == 'F',
                     data_melt.variable.str.split('_').str[0] == 'G',
                     data_melt.variable.str.split('_').str[0] == 'H']
    choicelist = ['G1','G1','G2','G2','G3','G3','G4','G4']
    data_melt['Group'] = np.select(conditionlist,choicelist, default='Not Specified')

    data_melt['value'] = data_melt['value'].round()
    data_melt['value'] = pd.to_numeric(data_melt['value'])
    data_melt['age'] = pd.to_numeric(data_melt['age'])
    
    return data_melt


TEnorm = TE_norm[TE_norm['TE_ID'].isin(DET_ALL['Row.names'].unique())]
TEnorm = pd.melt(TEnorm, id_vars=['TE_ID'])
TE_SQ = file_parsing(TEnorm)

TE_SQ['SUB1'] = TE_SQ['TE_ID'].str.split('.').str[5]
TE_SQ['SUB2'] = TE_SQ['TE_ID'].str.split('.').str[4]

In [ ]:
from pylab import *
rc('axes', linewidth=2)

fig = plt.figure(figsize=(13,5))

for N,i in enumerate(['SINE','LINE','LTR']):  
    
    SEL = TE_SQ[TE_SQ['SUB1'] == i]

    grouped_SEL = SEL['value'].groupby(SEL['variable'])
    GTE = pd.DataFrame(grouped_SEL.sum())
    GTE = GTE.reset_index(drop=False)
    GTE = file_parsing(GTE)
    
    age = GTE['age']
    value = GTE['value']
    r, p = stats.pearsonr(age,value)
   
    globals()["ax{}".format(N)] = fig.add_subplot(1,3,N+1)
    

    sns.regplot(data = GTE, x= 'age', y= 'value', ax=globals()["ax{}".format(N)])
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    plt.title(i,size=30)
    globals()["ax{}".format(N)].set_xlabel('Age',fontdict={'fontsize':15})
    
    globals()["ax{}".format(N)].text(0.95, 0.1,"R:{:3.2f}".format(r), fontsize =20, horizontalalignment='right',verticalalignment='bottom',transform=globals()["ax{}".format(N)].transAxes)
    globals()["ax{}".format(N)].text(0.95, 0.02, "P:{:3.5f}".format(p), fontsize =20, horizontalalignment='right',verticalalignment='bottom',transform=globals()["ax{}".format(N)].transAxes)

    sns.despine(right=True, top=True, ax=globals()["ax{}".format(N)])
    
    if N == 0:
        #globals()["ax{}".format(N)].set(ylabel = 'Normalized reads')
        
        globals()["ax{}".format(N)].set_ylabel('Normalized reads',fontdict={'fontsize':15})

    else:
        globals()["ax{}".format(N)].set(ylabel = '')

In [ ]:
from pylab import *
rc('axes', linewidth=2)

fig = plt.figure(figsize=(15,11))

for N,i in enumerate(['Alu','MIR','L1','L2','ERV1','ERVL','ERVK','LTR']):   
    
    SEL = TE_SQ[TE_SQ['SUB2'] == i]

    grouped_SEL = SEL['value'].groupby(SEL['variable'])
    GTE = pd.DataFrame(grouped_SEL.sum())
    GTE = GTE.reset_index(drop=False)
    GTE = file_parsing(GTE)
    
    age = GTE['age']
    value = GTE['value']
    r, p = stats.pearsonr(age,value)
    
    globals()["ax{}".format(N)] = fig.add_subplot(2,4,N+1)

    globals()["ax{}".format(N)] = sns.regplot(data = GTE, x= 'age', y= 'value')
    plt.yticks(fontsize=10)
    plt.xticks(fontsize=10)
    plt.title(i)
    
    globals()["ax{}".format(N)].text(0.95, 0.07,"pearsonR:{:3.2f}".format(r), horizontalalignment='right',verticalalignment='bottom',transform=globals()["ax{}".format(N)].transAxes)
    globals()["ax{}".format(N)].text(0.95, 0.02, "pvalue:{:3.10f}".format(p), horizontalalignment='right',verticalalignment='bottom',transform=globals()["ax{}".format(N)].transAxes)

    sns.despine(right=True, top=True, ax=globals()["ax{}".format(N)])
    
    if N == 0 or N == 4:
        globals()["ax{}".format(N)].set(ylabel = 'Normalized reads')
    else:
        globals()["ax{}".format(N)].set(ylabel = '')

In [ ]:
DET_G2['TEs'] = DET_G2.gene.str.split(':').str[2]
SUB1           = DET_G2[['TEs']]

DET_G2['sub2'] = DET_G2.gene.str.split(':').str[1]
SUB2           = DET_G2[['TEs','sub2']]

DET_G2['sub3'] = DET_G2.gene.str.split(':').str[1]
SUB3           = DET_G2[['TEs','sub3']]

#for g,i in zip(['G1','G2','G3','G4','G5','G6'],[DET_G1,DET_G2,DET_G3,DET_G4,DET_G5,DET_G6]):
for g,i in zip(['G2','G4'],[DET_G2,DET_G4]):

    i['TEs'] = i.gene.str.split(':').str[2]
    i['sub2'] = i.gene.str.split(':').str[1]
    i['sub3'] = i.gene.str.split(':').str[0]
    i = i[i['TEs'].isin(['LINE','SINE','LTR','DNA'])]

    #SUB1
    sub1 = i['TEs'].value_counts()
    sub1 = sub1.reset_index(drop=False)
    sub1.columns = ['TEs','counts']
    sub1 = sub1.rename(columns={'counts':g})
    sub1 = sub1.drop_duplicates()

    #SUB2
    sub2_value = i['sub2'].value_counts()
    sub2_value = sub2_value.reset_index(drop=False)
    sub2_value.columns = ['sub2','counts']

    sub2 = pd.merge(i,sub2_value,how='right', on='sub2')
    sub2 = sub2[['TEs','sub2','counts']]
    sub2 = sub2.rename(columns={'counts':g})
    sub2 = sub2.drop_duplicates()

    #SUB3
    sub3_value = i['sub3'].value_counts()
    sub3_value = sub3_value.reset_index(drop=False)
    sub3_value.columns = ['sub3','counts']

    sub3 = pd.merge(i,sub3_value,how='right', on='sub3')
    sub3 = sub3[['TEs','sub3','counts']]
    sub3 = sub3.rename(columns={'counts':g})
    sub3 = sub3.drop_duplicates()
    
    #SUB1,2,3 append    
    SUB1 = pd.merge(SUB1,sub1,how='outer', on=['TEs'])
    SUB2 = pd.merge(SUB2,sub2,how='outer', on=['TEs','sub2']) 
    SUB3 = pd.merge(SUB3,sub3,how='outer', on=['TEs','sub3'])
    
SUB1 = SUB1.drop_duplicates()
SUB2 = SUB2.drop_duplicates()
SUB3 = SUB3.drop_duplicates()

In [ ]:
SUB1 = SUB1.set_index('TEs')
SUB2 = SUB2.set_index('sub2')
SUB3 = SUB3.set_index('sub3')

In [ ]:
SUB1 = SUB1.replace(np.nan, 0)
SUB2 = SUB2.replace(np.nan, 0)
SUB3 = SUB3.replace(np.nan, 0)

In [ ]:
SUB1 = SUB1.sort_values(by='G4', ascending=False)
SUB1 = SUB1.iloc[:4,:]
SUB2 = SUB2.sort_values(by='G4', ascending=False)
SUB2 = SUB2.iloc[:10,:]
SUB3 = SUB3.sort_values(by='G4', ascending=False)
SUB3 = SUB3.iloc[:20,:]

In [ ]:
SUB1['TEs'] = SUB1.index+'_TEs'
SUB  = pd.concat([SUB1,SUB2],axis=0)

## Fig. 7c

In [ ]:
fig = plt.figure(figsize = (6,6))

gs = gridspec.GridSpec(nrows=17, ncols=5, height_ratios=[1]*4+[0.05]+[1]*10+[0.5,0.5],width_ratios=[2,2,1,2,2],wspace=0.7,hspace=0.3)

ax1 = plt.subplot(gs[16,1:4])
plt.xticks([])
plt.yticks([])
plt.text(0.5, 0, 'Number of TEs', ha='center', va='center', size=10)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax2 = plt.subplot(gs[0:4,0:2])
plt.text(250, 3, 'Pattern 2', ha='center', va='center', size=15)
plt.ylim(1,0)
plt.xlim(0,400)

ax3 = plt.subplot(gs[5:15,0:2])
plt.ylim(1,0)
plt.xlim(0,400)

ax4 = plt.subplot(gs[0:4,3:5])
plt.text(250, 3, 'Pattern 4', ha='center', va='center', size=15)
plt.ylim(1,0)
plt.xlim(0,400)

ax5 = plt.subplot(gs[5:15,3:5])
plt.ylim(1,0)
plt.xlim(0,400)

g=0
for i in SUB1.index:
    #globals()['ax{}'.format(n)] = plt.subplot(gs[g,2])
    plt.subplot(gs[g,2])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.35, i, ha='center', va='center', size=15)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    g+=1
    
g=5
for i in SUB2.index:
    #globals()['ax{}'.format(n)] = plt.subplot(gs[g,2])
    plt.subplot(gs[g,2])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, i, ha='center', va='center', size=10)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    g+=1


col = ['skyblue' if TE == 'SINE' 
        else 'pink' if TE == 'LINE'
        else 'navajowhite' if TE == 'LTR'
        else 'lightgreen' if TE == 'DNA'
        else 'green' for TE in SUB2['TEs']]


sns.barplot(data = SUB1, x= 'G2', y= SUB1.index,palette=["skyblue","pink","navajowhite","lightgreen"], ax=ax2)
ax2.invert_xaxis()
ax2.axes.get_yaxis().set_visible(False)
ax2.axes.get_xaxis().set_visible(False)
ax2.set(xlabel=None)

sns.barplot(data = SUB2, x= 'G2', y= SUB2.index,palette=col, ax=ax3)
ax3.invert_xaxis()
ax3.axes.get_yaxis().set_visible(False)
ax3.set(xlabel=None)

sns.barplot(data = SUB1, x= 'G4', y= SUB1.index,palette=["skyblue","pink","navajowhite","lightgreen"], ax=ax4)
ax4.axes.get_yaxis().set_visible(False)
ax4.axes.get_xaxis().set_visible(False)
ax4.set(xlabel=None)

sns.barplot(data = SUB2, x= 'G4', y= SUB2.index,palette=col, ax=ax5)
ax5.axes.get_yaxis().set_visible(False)
ax5.set(xlabel=None)

plt.show()

## Fig. 7d

In [ ]:
fig = plt.figure(figsize = (4,5))

gs = gridspec.GridSpec(nrows=22, ncols=5, height_ratios=[1]*20+[0.5,0.5],width_ratios=[2,2,3.5,2,2],wspace=1.5,hspace=0.3)

ax1 = plt.subplot(gs[21,1:4])
plt.xticks([])
plt.yticks([])
plt.text(0.5, -2, 'Number of TEs', ha='center', va='center', size=10)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

ax2 = plt.subplot(gs[0:20,0:2])
plt.text(25, -1.5, 'Pattern 2', ha='center', va='center', size=15)
plt.ylim(1,0)
plt.xlim(0,60)

ax3 = plt.subplot(gs[0:20,3:5])
plt.text(25, -1.5, 'Pattern 4', ha='center', va='center', size=15)
plt.ylim(1,0)
plt.xlim(0,60)

n=4
g=0
for i in SUB3.index:
    globals()['ax{}'.format(n)] = plt.subplot(gs[g,2])
    plt.xticks([])
    plt.yticks([])
    plt.text(0.5, 0.5, i, ha='center', va='center', size=10)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    n+=1
    g+=1


col = ['skyblue' if TE == 'SINE' 
        else 'pink' if TE == 'LINE'
        else 'navajowhite' if TE == 'LTR'
        else 'lightgreen' for TE in SUB3['TEs']]
    
    
sns.barplot(data = SUB3, x= 'G2', y= SUB3.index,palette=col, ax=ax2)
ax2.invert_xaxis()
ax2.axes.get_yaxis().set_visible(False)
ax2.set(xlabel=None)

sns.barplot(data = SUB3, x= 'G4', y= SUB3.index,palette=col, ax=ax3)
ax3.axes.get_yaxis().set_visible(False)
ax3.set(xlabel=None)

plt.show()



In [ ]:
DET_ALL['TEs'] = DET_ALL.gene.str.split(':').str[2]
DET_ALL['sub2'] = DET_ALL.gene.str.split(':').str[1]
DET_ALL['sub3'] = DET_ALL.gene.str.split(':').str[0]

DET_G24['TEs'] = DET_G24.gene.str.split(':').str[2]
DET_G24['sub2'] = DET_G24.gene.str.split(':').str[1]
DET_G24['sub3'] = DET_G24.gene.str.split(':').str[0]

In [ ]:
DET_ALL['chr'] = DET_ALL['Row.names'].str.split('.').str[0]
DET_ALL['group'] = DET_ALL['GROUP'].str.split('G').str[1]
DET_ALL['chr_n'] = DET_ALL['chr'].str.split('chr').str[1]
DET_ALL = DET_ALL[DET_ALL['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                       'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'])]

DET_G24['chr'] = DET_G24['Row.names'].str.split('.').str[0]
DET_G24['loc'] = DET_G24['Row.names'].str.split('.').str[1]
DET_G24['chr_n'] = DET_G24['chr'].str.split('chr').str[1]
DET_G24['group'] = DET_G24['GROUP'].str.split('G').str[1]
DET_G24 = DET_G24[DET_G24['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                       'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'])]

In [ ]:
DET_ALL['group'] = DET_ALL['group'].astype('int')
DET_ALL['chr_n'].replace('X', '21', inplace=True)
DET_ALL['chr_n'] = DET_ALL['chr_n'].astype('int')

DET_G24['group'] = DET_G24['group'].astype('int')
DET_G24['loc'] = DET_G24['loc'].astype('int')
DET_G24['chr_n'].replace('X', '21', inplace=True)
DET_G24['chr_n'] = DET_G24['chr_n'].astype('int')

In [ ]:
DET_ALL = DET_ALL.sort_values(by=['chr_n'], ascending=[True])
DET_ALL = DET_ALL.reset_index(drop=True)

In [ ]:
DET_G24 = DET_G24.sort_values(by=['chr_n','loc'], ascending=[True,True])
DET_G24 = DET_G24.reset_index(drop=True)
DET_G24 = DET_G24.reset_index(drop=False)

## Supplementary Fig. 8c

In [ ]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles



plt.figure(figsize=(5,5))
Course1=DETsum_ALL['gene']
Course2=DETT_ALL['gene']

vd = venn2([set(Course1), set(Course2)],set_labels=('SQuIRE','TEtranscript'),
            set_colors=('#F4ACB7','#c4e6ff'),
            alpha=0.8
            )
venn2_circles([set(Course1), set(Course2)], linestyle='-', linewidth=1, color='grey')
plt.show()

## Supplementary Fig. 8d

In [ ]:
DETsum_ALL_index = DETsum_ALL.rename(columns={'gene':'name'})
DETT_ALL_index= DETT_ALL.rename(columns={'gene':'name'})
intersected_df = pd.merge(DETsum_ALL_index,DETT_ALL_index, how='inner',on=['name'])
plt.figure(figsize = (10,10))
a = sns.lmplot(data=intersected_df, x='log2FoldChange_x', y='log2FoldChange_y',legend=False)
a.set(ylabel = 'log2FC_TEtranscript')
a.set(xlabel = 'log2FC_SQuIRE')
plt.show()

In [ ]:
rmsk_set = rmsk[rmsk['TEs'].isin(['LINE','SINE','LTR','DNA'])]
DET_ALL_set = DET_ALL[DET_ALL['TEs'].isin(['LINE','SINE','LTR','DNA'])]

In [ ]:
rmsk_set = rmsk_set[rmsk_set['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                  'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'])]

In [ ]:
ncbiRefSeq_tcpt = ncbiRefSeq[ncbiRefSeq['feature']=='transcript']
refGene_tcpt = refGene[refGene['feature']=='transcript']
ncbiRefSeq_tcpt = ncbiRefSeq_tcpt[ncbiRefSeq_tcpt['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                  'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'])]
refGene_tcpt = refGene_tcpt[refGene_tcpt['chr'].isin(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                                  'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'])]

In [ ]:
DET_ALL_set_G2 = DET_ALL_set[DET_ALL_set['GROUP']=='G2']
DET_ALL_set_G4 = DET_ALL_set[DET_ALL_set['GROUP']=='G4']

rmsk_count = (rmsk_set.groupby(['chr','TEs']).size()).reset_index()
ncbiRefSeq_tcpt_count = (ncbiRefSeq_tcpt.groupby('chr').size()).reset_index()
DET_ALL_count_G2 =(DET_ALL_set_G2.groupby(['chr','TEs']).size()).reset_index()
DET_ALL_count_G4 =(DET_ALL_set_G4.groupby(['chr','TEs']).size()).reset_index()

rmsk_count = rmsk_count.rename(columns={0:'rmsk_count'})
ncbiRefSeq_tcpt_count = ncbiRefSeq_tcpt_count.rename(columns={0:'gene_count'})
DET_ALL_count_G2 = DET_ALL_count_G2.rename(columns={0:'DET2_count'})
DET_ALL_count_G4 = DET_ALL_count_G4.rename(columns={0:'DET4_count'})

In [ ]:
chr_count = reduce(lambda left, right: pd.merge(left , right,on = ['chr','TEs'],how='outer'),
                   [rmsk_count,DET_ALL_count_G2,DET_ALL_count_G4])

chr_count = reduce(lambda left, right: pd.merge(left , right,on = ['chr'],how='outer'),
                   [chr_count,chr_size,ncbiRefSeq_tcpt_count])

In [ ]:
chr_count['rmsk_ratio'] = (chr_count['rmsk_count']/chr_count['size'])*1000000
chr_count['gene_ratio'] = (chr_count['gene_count']/chr_count['size'])*1000000
chr_count['DET2_ratio'] = (chr_count['DET2_count']/chr_count['size'])*1000000
chr_count['DET4_ratio'] = (chr_count['DET4_count']/chr_count['size'])*1000000
chr_count['genome_size'] = chr_count['size']/1000000

In [ ]:
chr_count['chr_n'] = chr_count['chr'].str.split('chr').str[1]
chr_count['chr_n'].replace('X', '21', inplace=True)
chr_count['chr_n'] = chr_count['chr_n'].astype('int')
chr_count = chr_count.sort_values(by=['chr_n'], ascending=[True])

## Fig. 7e

In [ ]:
fig = plt.figure(figsize = (10,7))

gs = gridspec.GridSpec(nrows=6, ncols=1, height_ratios=[4,1,1,1,1,1],hspace=0.2)


##ax1
ax1 = plt.subplot(gs[0,0])

col = ['darkgrey','black','darkgrey','black','darkgrey','black','darkgrey','black','darkgrey','black',
       'darkgrey','black','darkgrey','black','darkgrey','black','darkgrey','black','darkgrey','black','darkgrey']

sns.set_theme(style="white",font_scale=0.8)
sns.scatterplot(x='index',y='log2FoldChange',size=15,data=DET_G24,hue='chr',palette=col,ax=ax1)
ax1.set_ylim(-8,8)
ax1.set_xlim(-20,1470)
ax1.set(xlabel =None)
plt.yticks(fontsize=8)
plt.xticks([])
ax1.get_legend().remove()

mark_value = 5
p_point_name = DET_G24.loc[pd.to_numeric(DET_G24['log2FoldChange']) >= mark_value]['sub3']
p_point_y = DET_G24.loc[pd.to_numeric(DET_G24['log2FoldChange']) >= mark_value]['log2FoldChange']
p_point_x = DET_G24.loc[pd.to_numeric(DET_G24['log2FoldChange']) >= mark_value]['index']

n_point_name = DET_G24.loc[pd.to_numeric(DET_G24['log2FoldChange']) <= -(mark_value)]['sub3']
n_point_y = DET_G24.loc[pd.to_numeric(DET_G24['log2FoldChange']) <= -(mark_value)]['log2FoldChange']
n_point_x = DET_G24.loc[pd.to_numeric(DET_G24['log2FoldChange']) <= -(mark_value)]['index']


for x,y,name in zip(p_point_x,p_point_y,p_point_name):
    if name == 'MacERV2_LTR2':
        ax1.annotate(name,xy=(x,y), xytext=(x+60,y+0.5),horizontalalignment = 'center',size=8,weight='bold')
    else:
        ax1.annotate(name,xy=(x,y), xytext=(x,y+0.4),horizontalalignment = 'center',size=8,weight='bold')
                         
for x,y,name in zip(n_point_x,n_point_y,n_point_name):
    ax1.annotate(name,xy=(x,y), xytext=(x,y-1),horizontalalignment = 'center',size=8,weight='bold')

##ax2
ax2 = plt.subplot(gs[1,0])
sns.set_theme(style="white",font_scale=0.8)

sns.barplot(data=chr_count, x="chr",y="genome_size",color='lightgrey',
            order=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                   'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'],
            ax=ax2)

ax2.set(ylabel = 'Genomic\nsize\n(Mb)')
ax2.set(xlabel =None)
plt.yticks(fontsize=8)
plt.xticks([])


##ax3
ax3 = plt.subplot(gs[2,0])
sns.set_theme(style="white",font_scale=0.8)

sns.barplot(data=chr_count, x="chr", y="gene_ratio",color='lightgrey',
            order=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                   'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'],
            ax=ax3)

ax3.set(ylabel = 'Genes\ncount\nper Mb')
ax3.set(xlabel =None)
plt.yticks(fontsize=8)
plt.xticks([])

##ax4
ax4 = plt.subplot(gs[3,0])
sns.set_theme(style="white",font_scale=0.8)

sns.barplot(data=chr_count, x="chr", y="rmsk_ratio", palette='Pastel1', hue="TEs",
            order=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                   'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'],
            hue_order=['LINE','SINE','LTR','DNA'],
            ax=ax4)

ax4.set(ylabel = 'TEs\ncount\nper Mb')
ax4.set(xlabel =None)
ax4.get_legend().remove()
plt.yticks(fontsize=8)
plt.xticks([])

##ax5
ax5 = plt.subplot(gs[4,0])
sns.set_theme(style="white",font_scale=0.8)

sns.barplot(data=chr_count, x="chr", y="DET2_ratio",palette='Pastel1', hue="TEs",
              order=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                     'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'],
              hue_order=['LINE','SINE','LTR','DNA'],
              ax=ax5)

ax5.set(ylabel = 'Pattern2\ncount\nper Mb')
ax5.set(xlabel =None)
ax5.get_legend().remove()
plt.yticks(fontsize=8)
plt.xticks([])

##ax6
ax6 = plt.subplot(gs[5,0])
sns.set_theme(style="white",font_scale=0.8)

sns.barplot(data=chr_count, x="chr", y="DET4_ratio",palette='Pastel1', hue="TEs",
              order=['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10',
                     'chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chrX'],
              hue_order=['LINE','SINE','LTR','DNA'],
              ax=ax6)

ax6.set(ylabel = 'Pattern4\ncount\nper Mb')
ax6.set(xlabel =None)
ax6.get_legend().remove()
plt.yticks(fontsize=8)
plt.xticks(fontsize=8)

plt.show()